In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
from time import sleep
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time 
from unidecode import unidecode
import json 
from thefuzz import process,fuzz
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


import requests

def getProxy():
    keydepip = '26191c8fd4436643e7e3488c5c0bb0e8'
    authen_ip = "59.153.238.246"
    url = f'http://api.fasthideproxy.com/socks5?apikey={keydepip}&ip={authen_ip}&changenow=1'
    print(url)
    response = requests.get(url, timeout=50).json()
    proxies = {}
    print(response)
    ip = ""
    if response['code'] == 1:
        ip = response['message']

    elif response['code'] == 0:
        doi = response['message'].split(' ')[1]
        
    return (ip)

def tmproxy():
    key = "8591a5d98ed1ce9f97a5a036ced3aefd"
    url = f'https://app.proxyno1.com/api/change-key-ip/{key}'
    getIP = requests.get(url, timeout=10).json()
    proxies = {}
    if getIP['status'] == 0:
        checkTime = getIP['message']

        print(checkTime)

    elif getIP['status'] == 1:
        res = requests.get(f'https://app.proxyno1.com/api/key-status/{key}').json()

        port = res['data']['proxy']['HTTP_IPv4']
        ip_allow = res['data']['proxy']['ip']

        ip = f'{ip_allow}:{port}'
        socks5_url = f'http://{ip}'

        print( f' proxy = {socks5_url}' )
    
        return socks5_url

        

def initDriver():
    # proxy_data = tmproxy()
    prox = Proxy()
    prox.proxy_type = ProxyType.MANUAL
    prox.socks_version = 5
    prox.socks_proxy = "http://hndc32.proxyno1.com:41636"
    WINDOW_SIZE = "1000,2000"
    chrome_options = Options()
    chrome_options.proxy = prox
    chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('disable-infobars')
    
    driver = webdriver.Chrome(options=chrome_options)
    return driver



def check_similar(q,obj1):
    temp = (process.extract(q, obj1["features"], scorer=fuzz.ratio))
    print(temp)
    return len(temp)

driver = initDriver()

from selenium.webdriver.common.keys import Keys


domain = 'https://www.tripadvisor.com.vn'

question  = "khách sạn giá dưới 3 triệu, có bể bơi trên tầng thượng, phòng hút thuốc lá, phục vụ bữa sáng, có quầy bar. Tôi thích uống rượu vang và đi tắm hơi nên muốn có nhà tắm hơi"


data  = open("data.txt", "r").read().split("\n")
result = []
count = 0
WAIT_TIME = 7

import re 

for item in data:
    if(count < 15):
        # try:
            target = domain + item
            # target = "https://www.tripadvisor.com.vn/Hotel_Review-g293925-d15113167-Reviews-Au_Lac_Charner_Hotel-Ho_Chi_Minh_City.html"
            print("[log] ", target)
            driver.get(target)
            
            # Dùng chroPath để lấy xpath
            name = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[1]/div[1]/div[3]/div[1]/div[2]/div[1]/h1[1]'))).text
            print(name)
            address = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[1]/div[1]/div[3]/div[1]/div[3]/div[1]/div[1]/a[1]/div[1]'))).text
            print(address)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            time.sleep(3)
            
            price = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[2]/div[1]/div[3]/div[1]/div[3]/div[3]'))).text
            
            price = re.findall("(?:\d{1,3}\.)+\d{3}\s", price)
            print(price)
            
            rating = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[2]/div[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[2]/div[1]/div[1]/span[1]'))).text
            print(rating)
            
            feature = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[2]/div[1]/div[5]/div[1]/div[1]/div[1]/div[1]/div[2]/div[2]'))).text
            print(feature)
            
            
            obj = {
                "name_hotel": name, 
                "address" : address,
                "rating" : rating,
                "price" : price, 
                "features" : feature
            }
            print(obj)
            sim = check_similar(question,obj)
            print(obj['name_hotel'], sim)
            result.append(obj)
            count += 1
        # except Exception as e:
        #     print(e)
        #     break
            
    else:
        count = 0
        driver.quit()
        driver = initDriver()






[log]  https://www.tripadvisor.com.vn/Hotel_Review-g293925-d25442757-Reviews-La_Siesta_Premium_Sai_Gon-Ho_Chi_Minh_City.html
La Siesta Premium Sai Gon
Số 1 trong 749 khách sạn tại Thành phố Hồ Chí Minh
['4.144.000 ', '4.144.000 ', '3.048.088 ', '3.609.024 ', '3.048.088 ']
5,0
Tiện nghi của khách sạn
Internet tốc độ miễn phí (WiFi)
Bể bơi
Trung tâm thể dục thẩm mỹ
Bữa sáng miễn phí
Xe đưa đón sân bay
Khu để hành lý
Nhân viên hỗ trợ tiền sảnh
Giặt khô
Hiển thị thêm
Tiện nghi trong phòng
Điều hòa nhiệt độ
Dịch vụ phòng
Quầy bar mini
Loại phòng
Phòng không hút thuốc
Phòng Suite
{'name_hotel': 'La Siesta Premium Sai Gon', 'address': 'Số 1 trong 749 khách sạn tại Thành phố Hồ Chí Minh', 'rating': '5,0', 'price': ['4.144.000 ', '4.144.000 ', '3.048.088 ', '3.609.024 ', '3.048.088 '], 'features': 'Tiện nghi của khách sạn\nInternet tốc độ miễn phí (WiFi)\nBể bơi\nTrung tâm thể dục thẩm mỹ\nBữa sáng miễn phí\nXe đưa đón sân bay\nKhu để hành lý\nNhân viên hỗ trợ tiền sảnh\nGiặt khô\nHi

KeyboardInterrupt: 

In [15]:
import re
name = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//body[1]/div[1]/main[1]/span[1]/div[4]/div[2]/div[1]/div[3]/div[1]/div[3]/div[3]'))).text
re.findall("(?:\d{1,3}\.)+\d{3}\s", name)

['4.144.000 ', '4.144.000 ', '3.048.088 ', '3.609.024 ', '3.048.088 ']

In [18]:
result

[{'name_hotel': 'La Siesta Premium Sai Gon',
  'address': 'Số 1 trong 749 khách sạn tại Thành phố Hồ Chí Minh',
  'rating': '5,0',
  'price': ['4.144.000 ',
   '4.144.000 ',
   '3.048.088 ',
   '3.609.024 ',
   '3.048.088 '],
  'features': 'Tiện nghi của khách sạn\nInternet tốc độ miễn phí (WiFi)\nBể bơi\nTrung tâm thể dục thẩm mỹ\nBữa sáng miễn phí\nXe đưa đón sân bay\nKhu để hành lý\nNhân viên hỗ trợ tiền sảnh\nGiặt khô\nHiển thị thêm\nTiện nghi trong phòng\nĐiều hòa nhiệt độ\nDịch vụ phòng\nQuầy bar mini\nLoại phòng\nPhòng không hút thuốc\nPhòng Suite'},
 {'name_hotel': 'Silverland Ben Thanh',
  'address': 'Số 2 trong 749 khách sạn tại Thành phố Hồ Chí Minh',
  'rating': '5,0',
  'price': ['3.349.630 ', '3.349.630 ', '3.251.111 ', '3.251.111 '],
  'features': 'Tiện nghi của khách sạn\nBãi đỗ xe miễn phí\nInternet tốc độ miễn phí (WiFi)\nBể bơi\nBữa sáng miễn phí\nXe đưa đón sân bay\nPhòng họp\nGiặt khô\nDịch vụ giặt là\nHiển thị thêm\nTiện nghi trong phòng\nĐiều hòa nhiệt

In [ ]:
# pip install thefuzz

In [19]:
import json
with open("hotels.json", "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)